In [17]:
%pylab inline
import pandas as pd
import json
import time
pd.options.display.max_rows = 10000
import regex as re

from geopy.geocoders import Nominatim
geolocator = Nominatim()

from geopy.exc import GeocoderTimedOut

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)
    
from mtranslate import translate

Populating the interactive namespace from numpy and matplotlib


C:\Users\Edi\Anaconda\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['e']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
sprev=""
sprev_re=""
a=["á","ă","â"]
e=["é"]
i=["í","î"]
o=["ö","ó","ő"]
u=["ü","ú","ű"]
s=["ș"]
t=["ț"]
s_letters=[a,e,i,o,u,s,t]
s_Letters=["a","e","i","o","u","s","t"]

In [3]:
def do_match(wt,ws):
    """ wt is the one in which we will search ws is the searched one"""
    if len(wt)>0:
        length=len(ws)
        return ((re.search(r'(%s){e<=1}'%ws,wt) and length>6) or (re.search(r'(%s)'%ws,wt) and length<=6 and length>=4))
    else: return False

In [4]:
def replace_place(string):
    to_replace=['balan', 'barot', 'csikszereda', 'gyergyo', 'keresztur', 'kezdi',
       'kolozsvar', 'kovaszna', 'parajd', 'regen', 'segesvar', 'szentgyorgy',
       'szovata', 'toplita', 'udvarhely', 'vasarhely', 'vlahica']
    replacement=['Bălan', 'Baraolt', 'Miercurea Ciuc', 'Gheorgheni', 'Cristuru Secuiesc', 'Târgu Secuiesc',
       'Cluj-Napoca', 'Covasna', 'Praid', 'Reghin', 'Sighisoara', 'Sfântu Gheorghe',
       'Sovata', 'Toplița, Harghita', 'Odorheiu Secuiesc', 'Târgu Mureș', 'Vlahița']
    for i in arange(len(to_replace)):
        if string==to_replace[i]: return replacement[i]
    return string

In [5]:
def make_re(string):
    if(isinstance(string, str)):
        for i in arange(0,len(s_letters)):
            for c in s_letters[i]:
                if(c in string): 
                    string=string.replace(c,s_Letters[i])
    return string

In [6]:
sprev=''
is_ro=False

s_cprev=''
cprev=''
is_c=False

In [7]:
def find_country(string):
    global sprev
    global is_ro
    global cprev
    global s_cprev
    global is_c
    if(isinstance(string, str)):
        for name in ["Romania","Románia","România"]:
            if do_match(string,name): 
                return "Romania"
        for name in ["Hungary","Magyarország","Magyarorszag"]:
            if do_match(string,name): 
                return "Hungary"
            
        if is_ro and do_match(string,sprev):
            return "Romania2"      
        else:
            found=False
            for name in RO_211:
                if isinstance(name,str) and do_match(string,name): 
                    is_ro=True
                    found=True
                    sprev=string
            if(found==False): is_ro=False
            else:
                return "Romania2"
            
        if is_c and do_match(string,cprev):
            return s_cprev     
        else:
            found=False
            for name in Countries:
                if isinstance(name,str) and do_match(string,name): 
                    is_c=True
                    found=True
                    cprev=string
                    s_cprev=name
            if(found==False): is_c=False
            else:
                return s_cprev
            

    else: return "none"

In [8]:
data=pd.DataFrame.from_dict(json.loads(open("../first.json").read()))[:5000]#only first 5000 but these get the location where they live
data2=pd.DataFrame.from_dict(json.loads(open("../data.json").read()))#all the data, birth location add
ro_211=pd.read_excel('romania_telepulesek_211.xlsx')
RO_211=[]
for i in arange(0,len(ro_211["Ro"])):
    #RO_211.append(ro_211.loc[i,"Ro"])
    #RO_211.append(make_re(ro_211.loc[i,"Ro"]))
    RO_211.append(ro_211.loc[i,"Hu"])
    RO_211.append(make_re(ro_211.loc[i,"Hu"]))
    
countries=pd.read_excel('countries.xlsx')
Countries=[]
for i in arange(0,len(countries["Country"])):
    Countries.append(countries.loc[i,"Country"])
    Countries.append(make_re(countries.loc[i,"Country"]))


In [9]:
data.groupby(by="livesin").describe()

id                \
                                                  count          mean   
livesin                                                                 
Aachen, Germany                                     1.0  11608.000000   
Aalborg, Denmark                                    1.0  11121.000000   
Aalen                                               1.0  13896.000000   
Aarhus, Denmark                                     1.0  13344.000000   
Abano Terme                                         1.0  12090.000000   
Abu Dhabi, United Arab Emirates                     1.0  13152.000000   
Agerskov, Sonderjylland, Denmark                    1.0  12031.000000   
Agyagfalva, Harghita, Romania                       2.0  12338.500000   
Ahaus                                               1.0  11448.000000   
Ajkarendek, Veszprem, Hungary                       1.0  13854.000000   
Ajnád, Harghita, Romania                            1.0   1374.000000   
Alba Iulia                                          3.0  12175.333333   
Albacete                                            1.0  13708.000000   
Albis, Covasna, Romania                             1.0  10606.000000   
Alexandria, Virginia                                1.0  12886.000000   
Alfalú, Harghita, Romania                           1.0  11757.000000   
Alsóbóldogfalva, Harghita, Romania                  3.0  13095.000000   
Alsónémedi                                          1.0  11310.000000   
Alsósófalva, Harghita, Romania                      3.0  11528.000000   
Altbach                                             1.0  13980.000000   
Alunis, Mures                                       1.0  11902.000000   
Alunis, Salaj, Romania                              1.0  12023.000000   
Amantea                                             1.0  12076.000000   
Amriswil, Switzerland                               1.0  13450.000000   
Amsterdam, Netherlands                              1.0  14298.000000   
Ancona, Italy                                       1.0  12187.000000   
Antwerp, Belgium                                    1.0  11919.000000   
Anzio, Italy                                        1.0  10451.000000   
Arad, Arad                                          4.0   9924.250000   
Arenas de San Pedro, Spain                          1.0  12125.000000   
Arkos                                               3.0  12747.333333   
Ashford, Kent, England                              1.0  12634.000000   
Assisi, Italy                                       1.0  10235.000000   
Astromeriti, Nicosia, Cyprus                        1.0  12473.000000   
Atherstone, Warwickshire                            1.0  12348.000000   
Atlanta, Georgia                                    2.0  12406.500000   
Augsburg, Germany                                   3.0   9419.000000   
Aulla                                               1.0  12033.000000   
Aurora, Ontario                                     1.0  13069.000000   
Babenhausen, Hessen, Germany                        1.0  11574.000000   
Bacel, Covasna, Romania                             5.0  11975.000000   
Bad Nenndorf                                        1.0  13376.000000   
Baden, Austria                                      1.0  14170.000000   
Baden-Baden, Germany                                1.0  14092.000000   
Baiersbronn                                         1.0  13853.000000   
Baile Tusnad                                        4.0  10099.000000   
Balaguer                                            1.0  10064.000000   
Balan, Salaj, Romania                              64.0  11693.218750   
Balatonalmádi                                       1.0  12138.000000   
Ballymena                                           1.0  10312.000000   
Balánbánya, Harghita, Romania                       9.0  10596.888889   
Balástya                                            1.0  10222.000000   
Band, Mures                           

In [10]:
data.sort_values(by="livesin",inplace=True)
start=time.time()
data["livesin_country"]=data["livesin"].apply(find_country)
end=time.time()

In [11]:
print(end-start)

166.3101818561554


find_country:1000 profiles in <42 s
find_country:5000 profiles in 200 s->more or less linear

In [12]:
data.groupby(by="livesin_country").count()

,birth,id,livesin,marriedsince,marriedto,name,other1,other2,other3,other4,other5,other6,school,schoolyear,studwhat,studwhere,studyear,workwhat,workwhere
livesin_country,,,,,,,,,,,,,,,,,,,
Austria,25,25,25,2,4,25,0,1,0,0,0,0,0,0,1,13,3,5,14
Belgium,9,9,9,1,1,9,2,0,0,0,0,0,0,0,0,3,1,1,3
Bulgaria,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
China,2,2,2,0,0,2,0,0,0,0,0,0,0,0,1,1,0,0,0
Colombia,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
Croatia,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
Cyprus,3,3,3,1,1,3,0,0,0,0,0,0,0,0,0,1,0,0,1
Czech Republic,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1
Denmark,13,13,13,1,2,13,0,0,0,0,0,0,0,0,2,6,2,2,5


In [18]:
data.sort_values(by="livesin")[["livesin","livesin_country"]]

,livesin,livesin_country
11608,"Aachen, Germany",Germany
11121,"Aalborg, Denmark",Denmark
13896,Aalen,None
13344,"Aarhus, Denmark",Denmark
12090,Abano Terme,None
13152,"Abu Dhabi, United Arab Emirates",United Arab Emirates
12031,"Agerskov, Sonderjylland, Denmark",Denmark
13408,"Agyagfalva, Harghita, Romania",Romania
11269,"Agyagfalva, Harghita, Romania",Romania
11448,Ahaus,None


In [ ]:
data["birth"]=data["birth"].apply(replace_place)
data2["birth"]=data2["birth"].apply(replace_place)

In [ ]:
start=time.time()
for i in [0,2]:#birth livesin
    data.sort_values(by=data.columns[i],inplace=True,ascending=False)
    p_lat, p_lon, located_p=give_coordinates2(data[data.columns[i]])
    data[data.columns[i]+"_p_lat"]=p_lat
    data[data.columns[i]+"_p_lon"]=p_lon
    data[data.columns[i]+"_located"]=located_p
end=time.time()

In [ ]:
print(end-start)

In [ ]:
def get_countries(places):
    x=[]
    for place in places:
        if(place!=None):
            place_split=re.split("/|, ",place)
            x.append(place_split[len(place_split)-1])
        else: x.append(None)
    return x

Megtalalom az orszagot es leforditom angolra

In [ ]:
column="livesin_located"
data["NAME"]=get_countries(data[column])
data.sort_values(by="NAME",inplace=True,ascending=False)
data["NAME"]=[translate_s3(name) for name in data["NAME"]]

In [ ]:
all_data=pd.concat([all_data,data])

In [ ]:
all_data2 = all_data.drop_duplicates(subset='id')

In [ ]:
all_data2.describe()

In [ ]:
all_data2.to_json("second.json")